In [1]:
import torch
import os
# os.chdir('/content/Economy-and-Population-Growth')

In [2]:
# !git pull origin
torch.cuda.is_available()
from DataGeneration import generate_multiple_series
from Gibbs.Steps import *
from Gibbs.Parameters import step1_params, step2_params

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
data = generate_multiple_series(150, 120, 10**4, 10**5, "gibbs.csv")
w = np.random.random(120)
w = w/np.sum(w)
w = torch.tensor(w, device=device)
n = 120
m = 25
l = 10
T = 150
q = 16
q_hat = q+15
burn_in = 1
draws = 3

In [4]:
def run_Gibbs(data, n, m, l, w, T, q, q_hat, burn_in, draws, device):
    X_i, F, S_m, sigma_m, Sigma_m, rho_m, K, J, lambdas, kappas, p_parameters, s_Da, A, Sigma_A, f_0, mu_m, mu_c, omega_squared, U, G, C, Y, R_hat = initialize(data, n, m, l, T, q, q_hat, device)
    I, ws, Delta = step1_params(q_hat, n, U, w, device)
    e = step2_params(n, I, device)
    for counter in range(draws):
        X_i, C, Sigma, mu_C, Y0, X = step1(X_i, U, w, ws, Delta, Y, C, G, J, F, mu_c, lambdas, n, q_hat, device)
        F, S_m, Sigma_F = step2(e, f_0, mu_m, sigma_m, Sigma_m, s_Da, Sigma_A, Sigma, Delta, mu_C, ws, q_hat, X, Y0, device)
        G, inv_g_j, inv_c_i = step3(omega_squared, lambdas, kappas,  mu_c, K, J, q_hat, m, n, C, U, device)
        U.H, inv_h_k = step4(lambdas, kappas, U, omega_squared, K, G, inv_g_j, l, q_hat, device)
        mu_c = step5(q_hat, lambdas, C, G, J, inv_c_i, n, device)
        lambdas.lambda_c_i = step6(omega_squared, kappas, U, p_parameters, mu_c, C, G, J, q_hat, n, device)
        lambdas.lambda_g_j = step7(omega_squared, p_parameters, kappas, U, G, m, K, q_hat, device)
        U.U_c, U.theta_c_i = step8(omega_squared, p_parameters, kappas, lambdas, U, X_i, F, mu_c, G, J, q_hat, n, device)
        U.U_g, U.theta_g_j = step9(omega_squared, p_parameters, kappas, lambdas, U, G, K, m, device)
        U.theta_h_k = step10(omega_squared, p_parameters, kappas, U, l, device)
        omega_squared = step11(n, m, l, kappas, lambdas, U, q_hat)
        kappas.kappa_c_i = step12(omega_squared, p_parameters, lambdas, U, q_hat, n, device)
        kappas.kappa_g_j = step13(omega_squared, p_parameters, lambdas, U, q_hat, m, device)
        kappas.kappa_h_k = step14(omega_squared, p_parameters, U, q_hat, l, device)
        J = step15(omega_squared, kappas, lambdas, C, G, U, n, m, mu_c, q_hat, device)
        K = step16(omega_squared, kappas, lambdas, G, U, m, l, device)
        f_0, mu_m = step17(F, Sigma_F, q_hat, device)
        s_Da, A = step18(F, S_m, Sigma_A, f_0, mu_m, q_hat, device)
        sigma_m = step19(S_m, Sigma_m, p_parameters, q_hat, device)
        rho_m, Sigma_m = step20(U, sigma_m, S_m, device)
        p_parameters.p_c_lambda = step21(lambdas, device)
        p_parameters.p_g_lambda = step22(lambdas, device)
        p_parameters.p_c_theta = step23(U, device)
        p_parameters.p_g_theta = step24(U, device)
        p_parameters.p_h_theta = step25(U, device)
        p_parameters.p_c_kappa = step26(kappas, device)
        p_parameters.p_g_kappa = step27(kappas, device)
        p_parameters.p_h_k = step28(kappas, device)
    print(lambdas.lambda_c_i)


In [5]:
run_Gibbs(data, n, m, l, w, T, q, q_hat, burn_in, draws, device)

tensor([0.4354, 0.1979, 0.3562, 0.2771, 0.0000, 0.5938, 0.9500, 0.1979, 0.4750,
        0.2771, 0.0000, 0.9104, 0.8313, 0.5542, 0.9104, 0.3958, 0.5542, 0.7917,
        0.2771, 0.9500, 0.9500, 0.1583, 0.3167, 0.6729, 0.5938, 0.7125, 0.8313,
        0.7521, 0.3958, 0.2375, 0.1187, 0.9500, 0.7125, 0.6729, 0.8313, 0.8313,
        0.5146, 0.3958, 0.9500, 0.4750, 0.6333, 0.5146, 0.7917, 0.9104, 0.1187,
        0.8313, 0.4354, 0.8313, 0.2375, 0.5542, 0.5542, 0.0396, 0.3167, 0.6333,
        0.2771, 0.6729, 0.1583, 0.8708, 0.3562, 0.6729, 0.7125, 0.0396, 0.2375,
        0.8313, 0.1583, 0.0396, 0.1583, 0.6729, 0.9500, 0.0396, 0.4354, 0.5146,
        0.3167, 0.3958, 0.0396, 0.3562, 0.4354, 0.9104, 0.1979, 0.7917, 0.2771,
        0.5146, 0.3167, 0.6729, 0.0792, 0.0792, 0.4750, 0.0396, 0.3562, 0.0000,
        0.7917, 0.6333, 0.9500, 0.2771, 0.5542, 0.4750, 0.5146, 0.5146, 0.9500,
        0.1583, 0.9104, 0.8708, 0.4750, 0.3562, 0.7917, 0.6729, 0.5146, 0.0000,
        0.3562, 0.8708, 0.6333, 0.8313, 